In [26]:
from emlib.iterlib import pairwise
from dataclasses import dataclass
from emlib.lib import print_table
from emlib.mus import *


class TurntableTrack:
    def __init__(self, kind='snd'):
        self.kind = kind
        self.events = []
    
    def add(self, time, event):
        self.events.append((time, event))
        self.events.sort()
        
    def at(self, time):
        if not self.events or time < self.events[0][0]:
            return None
        for ev0, ev1 in pairwise(self.events):
            if ev0[0] <= time < ev1[0]:
                return ev0
        return self.events[-1]
    
    def __repr__(self):
        ls = []
        for ev in self.events:
            l = f"{ev[0]}: {ev[1]}"
            ls.append(l)
        return "\n".join(ls)
    
    def report(self):
        rows = []
        for time, descr in self.events:
            if ":" in descr:
                label, rest = descr.split(":")
            else:
                label = descr
                rest = "-"
            if self.kind == 'snd':
                diff = checksnd(rest)
                row = (label, rest, diff, time)
            else:
                row = (label, rest, time)
            rows.append(row)
        print_table(rows)
        
    def clear(self):
        self.events = []     
    
class TurntableState:
    def __init__(self):
        self.weight = TurntableTrack('weight')
        self.snd = TurntableTrack('snd')
        self.tracks = (self.weight, self.snd)
        
    def addWeight(self, time, descr):
        self.weight.add(time, descr)
    
    def addSnd(self, time, descr):
        self.snd.add(time, descr)
        
    def at(self, time):
        return [track.at(time) for track in self.tracks]
    
    def clear(self):
        for track in self.tracks:
            track.clear()
    

In [27]:
t1, t2, t3, t4 = [TurntableState() for _ in range(4)]
ts = [t1, t2, t3, t4]

def stateAt(time):
    rows = []
    for i, t in enumerate(ts):
        w, s = t.at(time)
        rows.append([i, w, s])
    print_table(rows)
    
def showchord(ttchord):
    ttchord.show()
    print(ttchord)
    ttchord.difftones().show()
    
def checksnd(descr):
    assert "/" in descr, descr
    snd45, snd33 = descr.split("/")
    ch45 = Chord(snd45)
    ch33 = Chord(snd33)
    if len(ch45) != len(ch33):
        raise ValueError("los acordes tienen diferente cantidad de notas")
    ch33teorico = ch45.transpose(r2i(33.3333/45))
    diff = sum(abs(n1.midi-n0.midi) for n0, n1 in zip(ch33teorico, ch33))
    return diff



## TT1

In [28]:
t1 = ts[0]
t1.clear()
t1.addSnd(1, "Snd1: 7E 7D+ / 6B-20 6A+30")
t1.addSnd(39, "Snd2: 8C 7B / 7G-20 7F#-20")
t1.addSnd(47, "Snd3: 7C# / 6G#-20")
t1.addWeight(70, "Pos1: 4Kg (small deviation)")
t1.addSnd(83, "Snd4: 7A+20 / 7E")
t1.addWeight(83, "No weight")
t1.addSnd(125, "Snd5: 3Bb-30 3Ab+20 / 3F- 3Eb")
t1.addWeight(147, "Pos1: 5Kg")
t1.addWeight(182, "No weight")
t1.addSnd(189, "Snd6: 4E 4D+ / 3B-20 3A+30")
t1.addWeight(203, "Pos1: 5Kg (-50c/+20c)")
t1.addSnd(218, "Snd7: 3F+ 3E / 3C+30 2B-20")
t1.addWeight(222, "Pos2: 2,5Kg")


## TT2

In [29]:
t2 = ts[1]
t2.clear()
t2.addSnd(1, "Snd1: 7B-30 7Bb-30 / 7F+ 7E+")
t2.addWeight(17, "Pos0: 5Kg")
t2.addWeight(23, "No weight")
t2.addSnd(34, "Snd2: 7Gb+20 7F / 7Db 7C-20")
t2.addSnd(47, "Snd3: 7D+ / 6A+30")
t2.addSnd(83, "Snd4: 7G#-30 / 7D+")
t2.addSnd(110, "Snd5: 4D 4C#+ / 3A-20 3G#+30")
t2.addSnd(124, "Snd6: 4Eb 4D- / 3Bb-20 3Ab+30")
t2.addWeight(124, "Alu")
t2.addSnd(183, "Snd7: 4A / 4E-20")
t2.addSnd(200, "Snd8: 4Eb / 3Bb-20")
t2.addSnd(219, "Snd9: 3B+20 / 3F#")


# TT3

In [30]:
t3 = ts[2]; t3.clear()
t3.addSnd(1, "Snd1: 7B-30 7Bb-30 / 7F+ 7E+")
t3.addSnd(23, "Snd2: 7D# 7C# / 6A#-20 6G#-20")
t3.addSnd(39, "Snd3: 7A 7G+ / 7E-20 7D+30")
t3.addSnd(47, "Snd4: 7D / 6A-20")
t3.addSnd(83, "Snd5: 4D 4C#+ / 3A-20 3G#+30")
t3.addWeight(112, "Pos0, 4Kg (??)")
t3.addSnd(124, "Snd6: 4Eb 4D- / 3Bb-20 3Ab+30")
t3.addWeight(135, "Pos1: 5Kg")
t3.addWeight(163, "Pos1: 4Kg (small deviation)")
t3.addSnd(189, "Snd7: 3B-20 3A+30 / 3F+60 3E+10")
t3.addWeight(189, "Pos1: 5Kg")

## TT4

In [31]:
t4 = ts[3]; t4.clear()
t4.addSnd(12, "Snd1: 3A 3G# / 3E-20 3D#-20")
t4.addSnd(23, "Snd2: 7A 7G+ / 7E-20 7D+30")
t4.addWeight(39, "Pos1: 4Kg (smalles noticiable dev)")
t4.addSnd(47, "Snd3: 7E-31 / 6B-")
t4.addWeight(63, "Pos1: 4Kg (small dev)")
t4.addSnd(83, "Snd4: 7D / 6A-20")
t4.addSnd(164, "Snd5: 3Bb-20 3Ab+30 / 3F-40 3Eb+10")
t4.addSnd(189, "Snd6: 4E 4D+ / 3B-20 3A+30")
t4.addWeight(207, "Pos0: 5Kg (almost no dev)")
t4.addWeight(214, "Pos1: 5Kg")
t4.addSnd(222, "Snd7: 3F+ 3E / 3C+30 2B-20")
t4.addWeight(222, "No weight")
t4.addWeight(236, "Pos1: 5Kg")

In [32]:
names = ["T1", "T2", "T3", "T4"]
for name, t in zip(names, (t1, t2, t3, t4)):
    print(name)
    t.snd.report()

T1


Snd1,7E 7D+ / 6B-20 6A+30,0.0089396,1
Snd2,8C 7B / 7G-20 7F#-20,0.0089396,39
Snd3,7C# / 6G#-20,0.0044698,47
Snd4,7A+20 / 7E,0.0044698,83
Snd5,3Bb-30 3Ab+20 / 3F- 3Eb,0.0089396,125
Snd6,4E 4D+ / 3B-20 3A+30,0.0089396,189
Snd7,3F+ 3E / 3C+30 2B-20,0.0089396,218


T2


Snd1,7B-30 7Bb-30 / 7F+ 7E+,0.0089396,1
Snd2,7Gb+20 7F / 7Db 7C-20,0.0089396,34
Snd3,7D+ / 6A+30,0.0044698,47
Snd4,7G#-30 / 7D+,0.0044698,83
Snd5,4D 4C#+ / 3A-20 3G#+30,0.0089396,110
Snd6,4Eb 4D- / 3Bb-20 3Ab+30,0.0089396,124
Snd7,4A / 4E-20,0.0044698,183
Snd8,4Eb / 3Bb-20,0.0044698,200
Snd9,3B+20 / 3F#,0.0044698,219


T3


Snd1,7B-30 7Bb-30 / 7F+ 7E+,0.0089396,1
Snd2,7D# 7C# / 6A#-20 6G#-20,0.0089396,23
Snd3,7A 7G+ / 7E-20 7D+30,0.0089396,39
Snd4,7D / 6A-20,0.0044698,47
Snd5,4D 4C#+ / 3A-20 3G#+30,0.0089396,83
Snd6,4Eb 4D- / 3Bb-20 3Ab+30,0.0089396,124
Snd7,3B-20 3A+30 / 3F+60 3E+10,0.0089396,189


T4


Snd1,3A 3G# / 3E-20 3D#-20,0.0089396,12
Snd2,7A 7G+ / 7E-20 7D+30,0.0089396,23
Snd3,7E-31 / 6B-,0.0055302,47
Snd4,7D / 6A-20,0.0044698,83
Snd5,3Bb-20 3Ab+30 / 3F-40 3Eb+10,0.0089396,164
Snd6,4E 4D+ / 3B-20 3A+30,0.0089396,189
Snd7,3F+ 3E / 3C+30 2B-20,0.0089396,222


In [33]:
names = ["T1", "T2", "T3", "T4"]
for name, t in zip(names, (t1, t2, t3, t4)):
    print(name)
    t.weight.report()

T1


Pos1,4Kg (small deviation),70
No weight,-,83
Pos1,5Kg,147
No weight,-,182
Pos1,5Kg (-50c/+20c),203
Pos2,"2,5Kg",222


T2


Pos0,5Kg,17
No weight,-,23
Alu,-,124


T3


"Pos0, 4Kg (??)",-,112
Pos1,5Kg,135
Pos1,4Kg (small deviation),163
Pos1,5Kg,189


T4


Pos1,4Kg (smalles noticiable dev),39
Pos1,4Kg (small dev),63
Pos0,5Kg (almost no dev),207
Pos1,5Kg,214
No weight,-,222
Pos1,5Kg,236


### Pesos

TT1:
    * 2.5 + 1.250 = 3.75 (~4)
    * 5 Kg
    * Total: 1.250, 2x2.5
    
TT2:
    * 5 Kg (2x2.5)
    * Aluminium + Magnet
    
TT3:
    * 2.5, 1.250, 5, 2.5
    * Total: 1.250, 2x2.5
    
TT4:
    * 2.5 + 1.250
    * 5
    * Total: 1.250, 2x2.5
    
Total:
    * 3 x 1.250
    * 8 x 2.5
    
Si se usan de 5Kg:
    * TT1: 2x2.5, 1x1.25
    * TT2: 5Kg
    * TT3: 1x2.5, 1x1.25, 1x5 o 1x2.5
    * TT4: 1x2.5, 1x1.25, 1x5 o 1x2.5
    * Total: 6x2.5, 1x5, 3x1.25